# **Analysis Summary**

**Core Method:**  

- Cosine Similarity

**Why Cosine Similarity?:** 

- 餘弦相似性可以非常直觀了解到任兩向量的相似程度。應用到推薦上時，也可以很直覺理解推薦對象與被推薦對象彼此之間的關聯性。




Products Recommend for CustomerID : 17841

In [ ]:
pd.DataFrame({"RecommendItem":[StockCD_Description_Map.get(x) for x in Cust_17841_Rec_List]})

,RecommendItem
0,SKULL AND CROSSBONES GARLAND
1,4 PEAR BOTANICAL DINNER CANDLES
2,GAOLERS KEYS DECORATIVE GARDEN
3,SWALLOW SQUARE TISSUE BOX
4,SET 3 PAPER VINTAGE CHICK PAPER EGG
5,GLASS APOTHECARY BOTTLE PERFUME
6,CLASSIC GLASS SWEET JAR
7,VINTAGE 2 METER FOLDING RULER
8,DOG LICENCE WALL ART
9,BICYCLE SAFTEY WALL ART


Products Recommend for CustomerID : 12433

In [ ]:
pd.DataFrame({"RecommendItem":[StockCD_Description_Map.get(x) for x in Cust_12433_Rec_List]})

,RecommendItem
0,CREAM HEART CARD HOLDER
1,REGENCY CAKESTAND 3 TIER
2,GREEN REGENCY TEACUP AND SAUCER
3,PINK REGENCY TEACUP AND SAUCER
4,ROSES REGENCY TEACUP AND SAUCER
5,PHOTO FRAME CORNICE
6,TRIPLE PHOTO FRAME CORNICE
7,FAMILY PHOTO FRAME CORNICE
8,12 MESSAGE CARDS WITH ENVELOPES
9,RECYCLED ACAPULCO MAT GREEN


Products Recommend for CustomerID : 15344

In [ ]:
pd.DataFrame({"RecommendItem":[StockCD_Description_Map.get(x) for x in Cust_15344_Rec_List]})

,RecommendItem
0,RED TOADSTOOL LED NIGHT LIGHT
1,RED HANGING HEART T-LIGHT HOLDER
2,LOVEBIRD HANGING DECORATION WHITE
3,HANGING METAL HEART LANTERN
4,HEART OF WICKER SMALL
5,WOODEN STAR CHRISTMAS SCANDINAVIAN
6,CARD HOLDER GINGHAM HEART
7,WICKER WREATH SMALL
8,RABBIT NIGHT LIGHT
9,CARD HOLDER LOVE BIRD SMALL


# **題目**

(2) 請推薦以下顧客，接下來可能購買的商品清單，理由是?
1. 17841
2. 12433
3. 15344

# **Import modules and data**

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [2]:
Valid_Transaction_Data = pd.read_csv("/content/Valid_Transaction_Data.csv")
Valid_Transaction_Data["InvoiceNo"] = Valid_Transaction_Data["InvoiceNo"].astype(str)
Valid_Transaction_Data["CustomerID"] = Valid_Transaction_Data["CustomerID"].astype(str)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Valid_Transaction_Data.shape

(530104, 25)

In [4]:
Valid_Transaction_Data["InvoiceNo_Int"].nunique() == Valid_Transaction_Data["InvoiceNo"].nunique()

True

# **Generate product purchase amount vector for each Customer**

I remove the Data with missing CustomerID since the clustering result doesn't contain those samples.

In [6]:
Valid_Transaction_Data_NoMiss = Valid_Transaction_Data[Valid_Transaction_Data["CustomerID"] != "Missing"]

In [7]:
StockCode_Vector_Data = Valid_Transaction_Data_NoMiss.pivot_table(index = ["CustomerID", "InvoiceNo_Int"], columns = "StockCode", values = "Quantity", aggfunc="sum", fill_value = 0).reset_index()
print("Data dim : ", StockCode_Vector_Data.shape)
print("Unique Customer Count : ", StockCode_Vector_Data["CustomerID"].nunique())

Data dim :  (18532, 3667)
Unique Customer Count :  4338


In [8]:
StockCode_Vector_Data.head()

StockCode,CustomerID,InvoiceNo_Int,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
0,12346,541431,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,12347,537626,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12347,542237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,12347,549222,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12347,556201,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18527,18283,579673,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18528,18283,580872,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18529,18287,554065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18530,18287,570715,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Create the vector of total amount on purchasing each product for every customer.

In [11]:
# Mean_ProductCnt_Vector_Df = StockCode_Vector_Data.drop("InvoiceNo_Int", axis = 1).groupby("CustomerID").mean()
Sum_ProductCnt_Vector_Df = StockCode_Vector_Data.drop("InvoiceNo_Int", axis = 1).groupby("CustomerID").sum()

In [12]:
Sum_ProductCnt_Vector_Df.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,15030,15034,15036,15039,15044A,15044B,15044C,15044D,15056BL,15056N,15056P,15058A,15058B,15058C,15060B,16008,16010,16011,16012,16014,16015,16016,16020C,16033,16043,16045,16046,16048,16049,16052,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12347,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12348,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
12349,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
12350,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# **Generate Dictionary for StockCode and Description**

In [24]:
temp = Valid_Transaction_Data_NoMiss.drop_duplicates("StockCode")[["StockCode", "Description"]]
StockCD_Description_Map = dict(zip(temp["StockCode"], temp["Description"]))

# **Define function for Recommendation**

- Using `Cosine Similarity`.

In [19]:
def Recommend(target_Customer_ID, Mean_data_OR_Sum_data):
  if type(target_Customer_ID) is not str : 
    target_Customer_ID = str(target_Customer_ID)
  df = Mean_data_OR_Sum_data.copy()
  df_values = df.values
  # Calculate cosine similarity
  cos_sin_mat = cosine_similarity(sparse.csr_matrix(df_values))
  cos_sin_DF = pd.DataFrame(cos_sin_mat, columns = df.index, index=df.index)
  target_cust = cos_sin_DF[target_Customer_ID]
  # Prevent from selecting itselves
  Aim_target_cust = target_cust[target_cust < 1]
  Highest_Cos_CustID_Cos = Aim_target_cust[Aim_target_cust == Aim_target_cust.max()]
  print(f"CustomerID : [{Highest_Cos_CustID_Cos.index[0]}] Has the highest cosine similarity :[{float(Highest_Cos_CustID_Cos.values):.4f}]")
  Highest_Cos_CustID = Highest_Cos_CustID_Cos.index[0]
  compare_df = pd.DataFrame({Highest_Cos_CustID : df.loc[Highest_Cos_CustID,:], target_Customer_ID :df.loc[target_Customer_ID,:]}, index = df.columns)
  compare_df["RecommendMark"] = compare_df[str(Highest_Cos_CustID)] - compare_df[target_Customer_ID]
  Recommend_Stock_Code = compare_df[compare_df["RecommendMark"] > 0].index
  print("Recommend List :", list(Recommend_Stock_Code))
  return list(Recommend_Stock_Code)

# **CustomerID : 17841**

In [20]:
Cust_17841_Rec_List = Recommend(17841, Sum_ProductCnt_Vector_Df)

CustomerID : [13573] Has the highest cosine similarity :[0.7834]
Recommend List : ['21207', '21616', '21745', '22097', '22957', '23012', '23169', '23188', '23526', '23535', '84596E', '84946']


product Recommend for CustomerID : 17841

In [26]:
pd.DataFrame({"RecommendItem":[StockCD_Description_Map.get(x) for x in Cust_17841_Rec_List]})

,RecommendItem
0,SKULL AND CROSSBONES GARLAND
1,4 PEAR BOTANICAL DINNER CANDLES
2,GAOLERS KEYS DECORATIVE GARDEN
3,SWALLOW SQUARE TISSUE BOX
4,SET 3 PAPER VINTAGE CHICK PAPER EGG
5,GLASS APOTHECARY BOTTLE PERFUME
6,CLASSIC GLASS SWEET JAR
7,VINTAGE 2 METER FOLDING RULER
8,DOG LICENCE WALL ART
9,BICYCLE SAFTEY WALL ART


# **CustomerID : 12433**

In [31]:
Cust_12433_Rec_List = Recommend(12433, Sum_ProductCnt_Vector_Df)

CustomerID : [14619] Has the highest cosine similarity :[0.4018]
Recommend List : ['22189', '22423', '22697', '22698', '22699', '22766', '22767', '22768', '22906', '23050', '23051', '23053', '23155']


product Recommend for CustomerID : 12433

In [32]:
pd.DataFrame({"RecommendItem":[StockCD_Description_Map.get(x) for x in Cust_12433_Rec_List]})

,RecommendItem
0,CREAM HEART CARD HOLDER
1,REGENCY CAKESTAND 3 TIER
2,GREEN REGENCY TEACUP AND SAUCER
3,PINK REGENCY TEACUP AND SAUCER
4,ROSES REGENCY TEACUP AND SAUCER
5,PHOTO FRAME CORNICE
6,TRIPLE PHOTO FRAME CORNICE
7,FAMILY PHOTO FRAME CORNICE
8,12 MESSAGE CARDS WITH ENVELOPES
9,RECYCLED ACAPULCO MAT GREEN


# **CustomerID : 15344**

In [22]:
Cust_15344_Rec_List = Recommend(15344, Sum_ProductCnt_Vector_Df)

CustomerID : [15692] Has the highest cosine similarity :[0.3750]
Recommend List : ['21731', '21733', '22219', '22464', '22469', '22578', '22588', '22695', '23084', '23469', '85123A']


product Recommend for CustomerID : 15344

In [28]:
pd.DataFrame({"RecommendItem":[StockCD_Description_Map.get(x) for x in Cust_15344_Rec_List]})

,RecommendItem
0,RED TOADSTOOL LED NIGHT LIGHT
1,RED HANGING HEART T-LIGHT HOLDER
2,LOVEBIRD HANGING DECORATION WHITE
3,HANGING METAL HEART LANTERN
4,HEART OF WICKER SMALL
5,WOODEN STAR CHRISTMAS SCANDINAVIAN
6,CARD HOLDER GINGHAM HEART
7,WICKER WREATH SMALL
8,RABBIT NIGHT LIGHT
9,CARD HOLDER LOVE BIRD SMALL
